# Алгоритм

## Описание проблемы

### **Что должен уметь алгоритм (вход-выход):**

На вход данные.


На выходе сформированный заказ, такой, что остаток в конце следующего дня в 10 вечера будет минимален.


### **Какие есть проблемы:**

1.   Заказ делается в 6 вечера, но магазин прекращает работу в 10 часов. Неизвестно точный остаток на конец дня.
2.   Что делать с новыми добавленными продуктами в систему?
3.   Какие гипотезы отражают реальную суть.

### **Какие возможные решения:**

1.   Предсказание остатков продуктов в 10 вечера на текущий день было предсказано в предыдущий день. Можно смотреть на разности (утро-6вечера), (утро-10вечера)
2.   Разделить продукты на информативные классы. Применять языковые модели для авто разметки или вручную вносить продукт в информативный класс






## Описание работы алгоритма




### **Задачи:**


1.   Предсказать количество проданного товара на следующий день по категориям/продуктам за счет фич и доп.фич
2.   Предсказать(рассчитать) остаток товаров на 10вечера текущего дня (кол-во проданного товара за день у нас есть с предсказания предыдущего дня)
3.   Предсказать(рассчитать) сколько нужно дозаказать товара на основе количества проданного товара в следующий день(п.1) и текущих остатков на текущий день в 10вечера +доп.фич, чтобы в конце следующего дня осталось минимальное число непроданных продуктов
------------------------------------------------------------------------------

### **На вход модели:**
Фичи (учитываются данные о товаре и данные о текущем дне):
1.   DAY_TYPE - (основная фича) тип дня (выходной, праздник, рабочий день)
2.   MONTH_TYPE - (основная фича) тип месяца (повышенный, не повышенный спрос)
3.   PRICE - (основная фича) цена на продукт
4.   PRODUCT_CLASS - (основная фича) класс продукта
5.   PRODUCT_INFORMATIVE_CLASS - (основная фича) информативный класс продукта
------------------------------------------------------------------------------
1.   PRODUCT_MORNING_COUNT - (возможная фича) количество товара утром (для каждого товара)
2.   PRODUCT_EVENINT_COUNT - (возможная фича) количество товара вечером (для каждого товара)
3.   PRODUCT_IN_ORDER_COUNT - (возможная фича) количество товара в заказе (для каждого товара)
4.   BUILDING_COUNT - (возможная фича) количество "особых" зданий рядом (для каждого типа зданий)
5.   WEATHER - (возможная фича) тип погоды в этот день

Целевая переменная:
1.   PRODUCT_DEMAND сколько товара было куплено за день.

## Реализация

### Импорт библиотек

In [1]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import catboost as ctb
import holidays
import matplotlib.pyplot as plt
import calendar, datetime
import random
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import RidgeClassifier
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation, SpectralClustering
from sklearn import utils

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score

### Генерация данных в соответствии с гипотезами и информацией о том, что на текущий момент хранится в системе компании

**На данный момент в системе компании хранятся следующие данные:**

*   Данные о составе заказов (какие товары, какое количество, цена товара)
*   Классификация товаров (хлебобулка, кондитерские изделия, свежезамороженные и др..)
*   Данные об остатках товара на конец дня
------------------------------------------------------------------------------
**ЦЕЛЕВАЯ ПЕРЕМЕННАЯ PRODUCT_DEMAND:**

Пусть PRODUCT_COUNT_DAY_START - количество такого-то товара на начало дня

Пусть PRODUCT_COUNT_DAY_END - количество такого-то товара на конец дня

**Можно выяснить спрос на товар в конкретный день по формуле:**
PRODUCT_COUNT_DAY_START - PRODUCT_COUNT_DAY_END

Так как реальных данных на текущий момент нет, то мы будем осуществлять генерацию датасета с использованием случайных данных(для придания данным более естественного вида) из отрезка PRODUCT_DEMAND.
Спрос на конкретный товар при каких-либо условиях будет корректироваться с помощью гипотез, описанных в этом .doc файле: https://docs.google.com/document/d/1uTw1Y6sYekTAKOGwAJ8jL8sztxxwSohsAYr6wgdQ9zs/edit?usp=sharing

------------------------------------------------------------------------------
**ФИЧИ PRICE, PRODUCT_CLASS, PRODUCT_INFORMATIVE_CLASS**

Такие данные, как цена на продукт, название продукта, его классификация внутри торговой сети нам известны.

В качестве фичи **PRODUCT_INFORMATIVE_CLASS** предлагается продумать информативный класс для групп продуктов. Например, "праздничные товары(торты, дорогие пирожные)", "продукты частого потребления(хлеб, полуфабрикаты)"

Автоматическую классификацию продуктов можно реализовать при помощи языковых моделей.

------------------------------------------------------------------------------
**ФИЧИ DAY_TYPE, MONTH_TYPE**

Данные фичи подразумевают под собой деление дней и месяцев на разные типы. Например, праздники, выходные, дни зарплаты.

В праздничные дни востребованность тортов увеличивается.

В дни зарплаты люди закупаются едой частого потребления.

Информация о типе конкретного дня обозначается в соответствии с открытыми источниками о праздниках и выходных.

------------------------------------------------------------------------------
**ГРУППА ФИЧ SPECIAL_BUILDING_COUNT**

Это возможная фича, отражающая количество особых зданий рядом с магазином.
Например, если рядом с магазином есть бизнес-центр или школа, то возможен повышенный спрос на булочки в рабочее время.



#### Технические операции, инициализации

In [3]:
# получение списка праздников в РФ
_hd = holidays.RU(2023)
_lhd = list(_hd)
_lhd.sort()

In [4]:
# форматирование даты и получение дней и месяцев
Y_M_D_main = pd.date_range('2023-01-01', '2023-12-31')
Y_M_D = Y_M_D_main.strftime("%y%m%d")
M_D = Y_M_D_main.strftime("%m%d")
list_Y_M_D = Y_M_D.to_list()
list_M_D = M_D.to_list()

#### Начальные известные данные

In [5]:
# список праздников
HOLIDAY_DAYS = [_lhd[i].strftime("%m%d") for i in range(len(_lhd))]
HOLIDAY_DAYS

['0101',
 '0102',
 '0103',
 '0104',
 '0105',
 '0106',
 '0107',
 '0108',
 '0223',
 '0224',
 '0308',
 '0501',
 '0508',
 '0509',
 '0612',
 '1104']

In [6]:
# список дней недели
days = Y_M_D_main.strftime("%d").to_list()
months = [datetime.datetime.strptime(list_Y_M_D[i], "%y%m%d").strftime('%B') for i in range(len(list_Y_M_D))]
week_days = [datetime.datetime.strptime(list_Y_M_D[i], "%y%m%d").strftime('%A') for i in range(len(list_Y_M_D))]

In [7]:
months

['January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'January',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'February',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'March',
 'Ma

In [8]:
# особые дни
INCREASED_DEMAND_MONTHS = ['11', '12']
SPECIAL_WEEK_DAYS = ['Monday', 'Wednesday', 'Thursday', 'Friday']
GETSALARY_DAYS = ['15', '30']
REDUCED_DEMAND_MONTHS = ['01', '06', '07', '08']
BAD_WEEK_DAYS = ['Saturday']

MONTH_NUMS = {'January': '01', 'February': '02', 'March': '03', 'April': '04', 'May': '05', 'June': '06', 'July': '07',
              'August': '08', 'September': '09', 'October': '10', 'November': '11', 'December': '12'}

WORK_DAYS = ['Monday', 'Wednesday', 'Thursday', 'Friday']

In [9]:
# наименование классов продуктов, информативных классов и имен продукта(участвующих в генерации искусственных данных)
INFORMATIVE_CLASSES = ['Продукты повышенного спроса', 'Продукты перекуса', 'Праздничные продукты']

CLASSES = ['Хлеб', 'Слойка', 'Пф', 'Торт', 'Пирожное', 'Сушка', 'Пирожок', 'Лепёшка']

PRODUCT_NAME = ['Хлеб Горчичный', 'Хлеб Гречишный', 'Хлеб Подольский', 'Хлеб Ситный', 'Хлеб Немецкий домашний',
                'Слойка Приморская с брусникой', 'Слойка Приморская с маком', 'Слойка Приморская с изюмом', 'Слойка Приморская с курагой',
                'Пф  штрудель вишня', 'Пф Булочка для гамбургера', 'Пф Минипицца Классика',
                'Торт Сгущенник', 'Торт Черепашка', 'Торт Морковный', 'Торт Праздничный', 'Торт Сказка сметанный',
                'Пирожное Блинное', 'Пирожное Эстерхази', 'Пирожное Морковное', 'Пирожное суфле Топленое молоко', 'Пирожное Александрия',
                'Сушки Кроха', 'Сушки Челночок',
                'Пирожок с нач из печени и риса ф', 'Пирожок с нач из капусты ф', 'Пирожок с нач из яйца и лука ф',
                'Лепёшка Сметанная', 'Лепёшка Ржаная', 'Лепёшка Гречишная']

In [10]:
# определение цен продуктов и коэффициентов спроса (для генерации искусственных данных)
n = len(PRODUCT_NAME)

PRODUCT_PRICE = np.zeros((n))
PRODUCT_PRICE[[PRODUCT_NAME.index('Хлеб Горчичный')]] = 52
PRODUCT_PRICE[[PRODUCT_NAME.index('Хлеб Гречишный')]] = 42
PRODUCT_PRICE[[PRODUCT_NAME.index('Хлеб Подольский')]] = 32
PRODUCT_PRICE[[PRODUCT_NAME.index('Хлеб Ситный')]] = 48
PRODUCT_PRICE[[PRODUCT_NAME.index('Хлеб Немецкий домашний')]] = 42
PRODUCT_PRICE[[PRODUCT_NAME.index('Слойка Приморская с брусникой')]] = 66
PRODUCT_PRICE[[PRODUCT_NAME.index('Слойка Приморская с маком')]] = 60
PRODUCT_PRICE[[PRODUCT_NAME.index('Слойка Приморская с изюмом')]] = 60
PRODUCT_PRICE[[PRODUCT_NAME.index('Слойка Приморская с курагой')]] = 62
PRODUCT_PRICE[[PRODUCT_NAME.index('Пф  штрудель вишня')]] = 180
PRODUCT_PRICE[[PRODUCT_NAME.index('Пф Булочка для гамбургера')]] = 29
PRODUCT_PRICE[[PRODUCT_NAME.index('Пф Минипицца Классика')]] = 79
PRODUCT_PRICE[[PRODUCT_NAME.index('Торт Сгущенник')]] = 519
PRODUCT_PRICE[[PRODUCT_NAME.index('Торт Черепашка')]] = 615
PRODUCT_PRICE[[PRODUCT_NAME.index('Торт Морковный')]] = 680
PRODUCT_PRICE[[PRODUCT_NAME.index('Торт Праздничный')]] = 725
PRODUCT_PRICE[[PRODUCT_NAME.index('Торт Сказка сметанный')]] = 625
PRODUCT_PRICE[[PRODUCT_NAME.index('Пирожное Блинное')]] = 150
PRODUCT_PRICE[[PRODUCT_NAME.index('Пирожное Эстерхази')]] = 159
PRODUCT_PRICE[[PRODUCT_NAME.index('Пирожное Морковное')]] = 120
PRODUCT_PRICE[[PRODUCT_NAME.index('Пирожное суфле Топленое молоко')]] = 55
PRODUCT_PRICE[[PRODUCT_NAME.index('Пирожное Александрия')]] = 110
PRODUCT_PRICE[[PRODUCT_NAME.index('Сушки Кроха')]] = 59
PRODUCT_PRICE[[PRODUCT_NAME.index('Сушки Челночок')]] = 75
PRODUCT_PRICE[[PRODUCT_NAME.index('Пирожок с нач из печени и риса ф')]] = 52
PRODUCT_PRICE[[PRODUCT_NAME.index('Пирожок с нач из капусты ф')]] = 45
PRODUCT_PRICE[[PRODUCT_NAME.index('Пирожок с нач из яйца и лука ф')]] = 48
PRODUCT_PRICE[[PRODUCT_NAME.index('Лепёшка Сметанная')]] = 32
PRODUCT_PRICE[[PRODUCT_NAME.index('Лепёшка Ржаная')]] = 27
PRODUCT_PRICE[[PRODUCT_NAME.index('Лепёшка Гречишная')]] = 24

PRODUCT_SECTION = np.zeros((n, 2))
PRODUCT_SECTION[[PRODUCT_NAME.index('Хлеб Горчичный')]] = [20, 35]
PRODUCT_SECTION[[PRODUCT_NAME.index('Хлеб Гречишный')]] = [15, 27]
PRODUCT_SECTION[[PRODUCT_NAME.index('Хлеб Подольский')]] = [27,  38]
PRODUCT_SECTION[[PRODUCT_NAME.index('Хлеб Ситный')]] = [12, 20]
PRODUCT_SECTION[[PRODUCT_NAME.index('Хлеб Немецкий домашний')]] = [24, 35]
PRODUCT_SECTION[[PRODUCT_NAME.index('Слойка Приморская с брусникой')]] = [28, 40]
PRODUCT_SECTION[[PRODUCT_NAME.index('Слойка Приморская с маком')]] = [26, 38]
PRODUCT_SECTION[[PRODUCT_NAME.index('Слойка Приморская с изюмом')]] = [20, 25]
PRODUCT_SECTION[[PRODUCT_NAME.index('Слойка Приморская с курагой')]] = [14, 22]
PRODUCT_SECTION[[PRODUCT_NAME.index('Пф  штрудель вишня')]] = [12, 20]
PRODUCT_SECTION[[PRODUCT_NAME.index('Пф Булочка для гамбургера')]] = [21, 32]
PRODUCT_SECTION[[PRODUCT_NAME.index('Пф Минипицца Классика')]] = [10, 12]
PRODUCT_SECTION[[PRODUCT_NAME.index('Торт Сгущенник')]] = [4, 7]
PRODUCT_SECTION[[PRODUCT_NAME.index('Торт Черепашка')]] = [6, 11]
PRODUCT_SECTION[[PRODUCT_NAME.index('Торт Морковный')]] = [5, 9]
PRODUCT_SECTION[[PRODUCT_NAME.index('Торт Праздничный')]] = [2, 5]
PRODUCT_SECTION[[PRODUCT_NAME.index('Торт Сказка сметанный')]] = [4, 7]
PRODUCT_SECTION[[PRODUCT_NAME.index('Пирожное Блинное')]] = [9, 13]
PRODUCT_SECTION[[PRODUCT_NAME.index('Пирожное Эстерхази')]] = [8, 14]
PRODUCT_SECTION[[PRODUCT_NAME.index('Пирожное Морковное')]] = [10, 13]
PRODUCT_SECTION[[PRODUCT_NAME.index('Пирожное суфле Топленое молоко')]] = [9, 14]
PRODUCT_SECTION[[PRODUCT_NAME.index('Пирожное Александрия')]] = [7, 12]
PRODUCT_SECTION[[PRODUCT_NAME.index('Сушки Кроха')]] = [14, 18]
PRODUCT_SECTION[[PRODUCT_NAME.index('Сушки Челночок')]] = [12, 16]
PRODUCT_SECTION[[PRODUCT_NAME.index('Пирожок с нач из печени и риса ф')]] = [30, 44]
PRODUCT_SECTION[[PRODUCT_NAME.index('Пирожок с нач из капусты ф')]] = [29, 37]
PRODUCT_SECTION[[PRODUCT_NAME.index('Пирожок с нач из яйца и лука ф')]] = [33, 42]
PRODUCT_SECTION[[PRODUCT_NAME.index('Лепёшка Сметанная')]] = [17, 22]
PRODUCT_SECTION[[PRODUCT_NAME.index('Лепёшка Ржаная')]] = [14, 19]
PRODUCT_SECTION[[PRODUCT_NAME.index('Лепёшка Гречишная')]] = [12, 16]

#[Стандарт=1, Суббота, Месяцы пониженного спроса, Дни получения зарплаты, Особые дни недели, Месяцы повышенного спроса, Праздники]

PRODUCT_COEFFICIENT = np.ones((n, 7))
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Хлеб Горчичный')]] = [1, 0.8 , 0.92, 1.08, 1.21, 1.25, 1.36]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Хлеб Гречишный')]] = [1, 0.85, 0.92, 1.08, 1.21, 1.24, 1.31]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Хлеб Подольский')]] = [1, 0.85, 0.82, 1.08, 1.21, 1.25, 1.36]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Хлеб Ситный')]] = [1, 0.8 , 0.92, 1.08, 1.21, 1.24, 1.29]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Хлеб Немецкий домашний')]] = [1, 0.85, 0.82, 1.08, 1.21, 1.24, 1.35]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Слойка Приморская с брусникой')]] = [1, 0.8 , 0.82, 1.13, 1.13, 1.16, 1.26]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Слойка Приморская с маком')]] = [1, 0.8 , 0.92, 1.13, 1.13, 1.17, 1.25]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Слойка Приморская с изюмом')]] = [1, 0.8 , 0.92, 1.13, 1.13, 1.15, 1.23]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Слойка Приморская с курагой')]] = [1, 0.85, 0.82, 1.13, 1.13, 1.16, 1.24]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Пф  штрудель вишня')]] = [1, 0.85, 0.82, 1.23, 1.24, 1.28, 1.32]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Пф Булочка для гамбургера')]] = [1, 0.8 , 0.82, 1.  , 1.03, 1.08, 1.13]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Пф Минипицца Классика')]] = [1, 0.85, 0.92, 1.03, 1.03, 1.06, 1.16]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Торт Сгущенник')]] = [1, 0.8 , 0.92, 1.15, 1.21, 1.3 , 1.53]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Торт Черепашка')]] = [1, 0.85, 0.82, 1.15, 1.21, 1.32, 1.58]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Торт Морковный')]] = [1, 0.8 , 0.82, 1.15, 1.21, 1.31, 1.55]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Торт Праздничный')]] = [1, 0.85, 0.92, 1.15, 1.21, 1.33, 1.63]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Торт Сказка сметанный')]] = [1, 0.85, 0.82, 1.15, 1.21, 1.3 , 1.59]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Пирожное Блинное')]] = [1, 0.85, 0.92, 1.12, 1.18, 1.28, 1.41]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Пирожное Эстерхази')]] = [1, 0.8 , 0.82, 1.12, 1.17, 1.26, 1.44]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Пирожное Морковное')]] = [1, 0.85, 0.92, 1.13, 1.17, 1.25, 1.38],
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Пирожное суфле Топленое молоко')]] = [1, 0.85, 0.92, 1.09, 1.14, 1.14, 1.29]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Пирожное Александрия')]] = [1, 0.8 , 0.92, 1.12, 1.17, 1.22, 1.31]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Сушки Кроха')]] = [1, 0.8 , 0.92, 1.05, 1.06, 1.09, 1.21]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Сушки Челночок')]] = [1, 0.85, 0.82, 1.05, 1.06, 1.08, 1.21]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Пирожок с нач из печени и риса ф')]] = [1, 0.8 , 0.82, 1.13, 1.18, 1.24, 1.36]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Пирожок с нач из капусты ф')]] = [1, 0.85, 0.82, 1.13, 1.16, 1.22, 1.35]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Пирожок с нач из яйца и лука ф')]] = [1, 0.85, 0.82, 1.13, 1.19, 1.24, 1.4]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Лепёшка Сметанная')]] = [1, 0.8 , 0.92, 1.08, 1.21, 1.23, 1.42]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Лепёшка Ржаная')]] = [1, 0.8 , 0.92, 1.08, 1.19, 1.22, 1.41]
PRODUCT_COEFFICIENT[[PRODUCT_NAME.index('Лепёшка Гречишная')]] = [1, 0.85, 0.92, 1.08, 1.19, 1.24, 1.41]

PRODUCT_TYPE = ["None" for i in range(n)]

PRODUCT_TYPE[PRODUCT_NAME.index('Хлеб Горчичный')] = CLASSES[CLASSES.index('Хлеб')]
PRODUCT_TYPE[PRODUCT_NAME.index('Хлеб Гречишный')] = CLASSES[CLASSES.index('Хлеб')]
PRODUCT_TYPE[PRODUCT_NAME.index('Хлеб Подольский')] = CLASSES[CLASSES.index('Хлеб')]
PRODUCT_TYPE[PRODUCT_NAME.index('Хлеб Ситный')] = CLASSES[CLASSES.index('Хлеб')]
PRODUCT_TYPE[PRODUCT_NAME.index('Хлеб Немецкий домашний')] = CLASSES[CLASSES.index('Хлеб')]
PRODUCT_TYPE[PRODUCT_NAME.index('Слойка Приморская с брусникой')] = CLASSES[CLASSES.index('Слойка')]
PRODUCT_TYPE[PRODUCT_NAME.index('Слойка Приморская с маком')] = CLASSES[CLASSES.index('Слойка')]
PRODUCT_TYPE[PRODUCT_NAME.index('Слойка Приморская с изюмом')] = CLASSES[CLASSES.index('Слойка')]
PRODUCT_TYPE[PRODUCT_NAME.index('Слойка Приморская с курагой')] = CLASSES[CLASSES.index('Слойка')]
PRODUCT_TYPE[PRODUCT_NAME.index('Пф  штрудель вишня')] = CLASSES[CLASSES.index('Пф')]
PRODUCT_TYPE[PRODUCT_NAME.index('Пф Булочка для гамбургера')] = CLASSES[CLASSES.index('Пф')]
PRODUCT_TYPE[PRODUCT_NAME.index('Пф Минипицца Классика')] = CLASSES[CLASSES.index('Пф')]
PRODUCT_TYPE[PRODUCT_NAME.index('Торт Сгущенник')] = CLASSES[CLASSES.index('Торт')]
PRODUCT_TYPE[PRODUCT_NAME.index('Торт Черепашка')] = CLASSES[CLASSES.index('Торт')]
PRODUCT_TYPE[PRODUCT_NAME.index('Торт Морковный')] = CLASSES[CLASSES.index('Торт')]
PRODUCT_TYPE[PRODUCT_NAME.index('Торт Праздничный')] = CLASSES[CLASSES.index('Торт')]
PRODUCT_TYPE[PRODUCT_NAME.index('Торт Сказка сметанный')] = CLASSES[CLASSES.index('Торт')]
PRODUCT_TYPE[PRODUCT_NAME.index('Пирожное Блинное')] = CLASSES[CLASSES.index('Пирожное')]
PRODUCT_TYPE[PRODUCT_NAME.index('Пирожное Эстерхази')] = CLASSES[CLASSES.index('Пирожное')]
PRODUCT_TYPE[PRODUCT_NAME.index('Пирожное Морковное')] = CLASSES[CLASSES.index('Пирожное')]
PRODUCT_TYPE[PRODUCT_NAME.index('Пирожное суфле Топленое молоко')] = CLASSES[CLASSES.index('Пирожное')]
PRODUCT_TYPE[PRODUCT_NAME.index('Пирожное Александрия')] = CLASSES[CLASSES.index('Пирожное')]
PRODUCT_TYPE[PRODUCT_NAME.index('Сушки Кроха')] = CLASSES[CLASSES.index('Сушка')]
PRODUCT_TYPE[PRODUCT_NAME.index('Сушки Челночок')] = CLASSES[CLASSES.index('Сушка')]
PRODUCT_TYPE[PRODUCT_NAME.index('Пирожок с нач из печени и риса ф')] = CLASSES[CLASSES.index('Пирожок')]
PRODUCT_TYPE[PRODUCT_NAME.index('Пирожок с нач из капусты ф')] = CLASSES[CLASSES.index('Пирожок')]
PRODUCT_TYPE[PRODUCT_NAME.index('Пирожок с нач из яйца и лука ф')] = CLASSES[CLASSES.index('Пирожок')]
PRODUCT_TYPE[PRODUCT_NAME.index('Лепёшка Сметанная')] = CLASSES[CLASSES.index('Лепёшка')]
PRODUCT_TYPE[PRODUCT_NAME.index('Лепёшка Ржаная')] = CLASSES[CLASSES.index('Лепёшка')]
PRODUCT_TYPE[PRODUCT_NAME.index('Лепёшка Гречишная')] = CLASSES[CLASSES.index('Лепёшка')]

PRODUCT_INFOTYPE = ["None" for i in range(n)]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Хлеб Горчичный')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Продукты повышенного спроса')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Хлеб Гречишный')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Продукты повышенного спроса')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Хлеб Подольский')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Продукты повышенного спроса')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Хлеб Ситный')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Продукты повышенного спроса')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Хлеб Немецкий домашний')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Продукты повышенного спроса')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Слойка Приморская с брусникой')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Продукты перекуса')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Слойка Приморская с маком')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Продукты перекуса')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Слойка Приморская с изюмом')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Продукты перекуса')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Слойка Приморская с курагой')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Продукты перекуса')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Пф  штрудель вишня')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Продукты перекуса')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Пф Булочка для гамбургера')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Продукты перекуса')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Пф Минипицца Классика')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Продукты перекуса')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Торт Сгущенник')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Праздничные продукты')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Торт Черепашка')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Праздничные продукты')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Торт Морковный')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Праздничные продукты')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Торт Праздничный')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Праздничные продукты')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Торт Сказка сметанный')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Праздничные продукты')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Пирожное Блинное')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Праздничные продукты')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Пирожное Эстерхази')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Праздничные продукты')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Пирожное Морковное')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Праздничные продукты')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Пирожное суфле Топленое молоко')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Праздничные продукты')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Пирожное Александрия')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Праздничные продукты')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Сушки Кроха')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Продукты перекуса')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Сушки Челночок')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Продукты перекуса')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Пирожок с нач из печени и риса ф')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Продукты перекуса')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Пирожок с нач из капусты ф')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Продукты перекуса')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Пирожок с нач из яйца и лука ф')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Продукты перекуса')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Лепёшка Сметанная')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Продукты повышенного спроса')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Лепёшка Ржаная')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Продукты повышенного спроса')]
PRODUCT_INFOTYPE[PRODUCT_NAME.index('Лепёшка Гречишная')] = INFORMATIVE_CLASSES[INFORMATIVE_CLASSES.index('Продукты повышенного спроса')]

#### Формирование датасета

**Датасет будет иметь следующий вид:**
Строка содержит в себе следующие данные:

Если данные за прошлые дни: [PRODUCT_NAME, PRICE, DAY, MONTH, PRODUCT_DEMAND]

Если данные за будущие дни: [PRODUCT_NAME, PRICE, DAY, MONTH]

In [11]:
main = [[PRODUCT_INFOTYPE[i], PRODUCT_TYPE[i], PRODUCT_NAME[i], PRODUCT_PRICE[i], PRODUCT_SECTION[i], PRODUCT_COEFFICIENT[i]] for i in range(n)]

In [12]:
data = []
for i in range(len(days)):
    if (list_M_D[i] in HOLIDAY_DAYS):
        data.append([[main[j][0], main[j][1], main[j][3], days[i], months[i], random.randint(round(main[j][4][0] * main[j][5][-1]), round((main[j][4][0] * main[j][5][-1]) + (main[j][4][1] - main[j][4][0])))] for j in range(n)])
    elif (months[i] in INCREASED_DEMAND_MONTHS):
        data.append([[main[j][0], main[j][1], main[j][3], days[i], months[i], random.randint(round(main[j][4][0] * main[j][5][-2]), round((main[j][4][0] * main[j][5][-2]) + (main[j][4][1] - main[j][4][0])))] for j in range(n)])
    elif (week_days[i] in SPECIAL_WEEK_DAYS):
        data.append([[main[j][0], main[j][1], main[j][3], days[i], months[i], random.randint(round(main[j][4][0] * main[j][5][-3]), round((main[j][4][0] * main[j][5][-3]) + (main[j][4][1] - main[j][4][0])))] for j in range(n)])
    elif (days[i] in GETSALARY_DAYS):
        data.append([[main[j][0], main[j][1], main[j][3], days[i], months[i], random.randint(round(main[j][4][0] * main[j][5][-4]), round((main[j][4][0] * main[j][5][-4]) + (main[j][4][1] - main[j][4][0])))] for j in range(n)])
    elif (months[i] in REDUCED_DEMAND_MONTHS):
        data.append([[main[j][0], main[j][1], main[j][3], days[i], months[i], random.randint(round(main[j][4][0] * main[j][5][-5]), round((main[j][4][0] * main[j][5][-5]) + (main[j][4][1] - main[j][4][0])))] for j in range(n)])
    elif (week_days[i] in BAD_WEEK_DAYS):
        data.append([[main[j][0], main[j][1], main[j][3], days[i], months[i], random.randint(round(main[j][4][0] * main[j][5][-6]), round((main[j][4][0] * main[j][5][-6]) + (main[j][4][1] - main[j][4][0])))] for j in range(n)])
    else:
        data.append([[main[j][0], main[j][1], main[j][3], days[i], months[i], random.randint(main[j][4][0], main[j][4][1])] for j in range(n)])
np.array(data)

array([[['Продукты повышенного спроса', 'Хлеб', '52.0', '01', 'January',
         '28'],
        ['Продукты повышенного спроса', 'Хлеб', '42.0', '01', 'January',
         '29'],
        ['Продукты повышенного спроса', 'Хлеб', '32.0', '01', 'January',
         '44'],
        ...,
        ['Продукты повышенного спроса', 'Лепёшка', '32.0', '01',
         'January', '27'],
        ['Продукты повышенного спроса', 'Лепёшка', '27.0', '01',
         'January', '25'],
        ['Продукты повышенного спроса', 'Лепёшка', '24.0', '01',
         'January', '17']],

       [['Продукты повышенного спроса', 'Хлеб', '52.0', '02', 'January',
         '32'],
        ['Продукты повышенного спроса', 'Хлеб', '42.0', '02', 'January',
         '26'],
        ['Продукты повышенного спроса', 'Хлеб', '32.0', '02', 'January',
         '39'],
        ...,
        ['Продукты повышенного спроса', 'Лепёшка', '32.0', '02',
         'January', '26'],
        ['Продукты повышенного спроса', 'Лепёшка', '27.0', '02',
     

In [13]:
TO_PANDAS = []
for i in range(n):
    for j in range(len(days)):
        TO_PANDAS.append(data[j][i])
LAST_DATA_CONVERTATION = np.array(TO_PANDAS)

df_main = pd.DataFrame(LAST_DATA_CONVERTATION)
df_main.columns = ['PRODUCT_INFORMATIVE_CLASS', 'PRODUCT_CLASS', 'PRICE', 'DAY', 'MONTH', 'PRODUCT_DEMAND']

In [14]:
df_main

,PRODUCT_INFORMATIVE_CLASS,PRODUCT_CLASS,PRICE,DAY,MONTH,PRODUCT_DEMAND
0,Продукты повышенного спроса,Хлеб,52.0,01,January,28
1,Продукты повышенного спроса,Хлеб,52.0,02,January,32
2,Продукты повышенного спроса,Хлеб,52.0,03,January,35
3,Продукты повышенного спроса,Хлеб,52.0,04,January,37
4,Продукты повышенного спроса,Хлеб,52.0,05,January,39
...,...,...,...,...,...,...
10945,Продукты повышенного спроса,Лепёшка,24.0,27,December,15
10946,Продукты повышенного спроса,Лепёшка,24.0,28,December,14
10947,Продукты повышенного спроса,Лепёшка,24.0,29,December,17
10948,Продукты повышенного спроса,Лепёшка,24.0,30,December,14


#### Подготовка данных для модели

**В модель будут подаваться следующие данные**

Фичи = [PRICE, PRODUCT_CLASS, PRODUCT_INFORMATIVE_CLASS, DAY_TYPE, MONTH_TYPE]

Целевая переменная = [PRODUCT_DEMAND]


In [15]:
day_types = []
for elem in range(len(df_main)):
  if MONTH_NUMS[df_main['MONTH'][elem]] + df_main['DAY'][elem] in HOLIDAY_DAYS:
    day_types.append('holiday')
  elif df_main['DAY'][elem] in GETSALARY_DAYS:
    day_types.append('salary')
  else:
    day_types.append('default')

month_types = []

for elem in range(len(df_main)):
  if MONTH_NUMS[df_main['MONTH'][elem]] in INCREASED_DEMAND_MONTHS:
    month_types.append('good')
  elif MONTH_NUMS[df_main['MONTH'][elem]] in REDUCED_DEMAND_MONTHS:
    month_types.append('bad')
  else:
    month_types.append('default')


X = pd.DataFrame()
X['PRODUCT_INFORMATIVE_CLASS'] = [INFORMATIVE_CLASSES.index(i) for i in df_main['PRODUCT_INFORMATIVE_CLASS']]
X['PRODUCT_CLASS'] = [CLASSES.index(i) for i in df_main['PRODUCT_CLASS']]
X['PRICE'] = df_main['PRICE']
X['DAY_TYPE'] = day_types
X['MONTH_TYPE'] = month_types
X.columns = ['PRODUCT_INFORMATIVE_CLASS', 'PRODUCT_CLASS', 'PRICE', 'DAY_TYPE', 'MONTH_TYPE']

Y = df_main['PRODUCT_DEMAND']

In [16]:
X

,PRODUCT_INFORMATIVE_CLASS,PRODUCT_CLASS,PRICE,DAY_TYPE,MONTH_TYPE
0,0,0,52.0,holiday,bad
1,0,0,52.0,holiday,bad
2,0,0,52.0,holiday,bad
3,0,0,52.0,holiday,bad
4,0,0,52.0,holiday,bad
...,...,...,...,...,...
10945,0,7,24.0,default,good
10946,0,7,24.0,default,good
10947,0,7,24.0,default,good
10948,0,7,24.0,salary,good


In [17]:
Y

0        28
1        32
2        35
3        37
4        39
         ..
10945    15
10946    14
10947    17
10948    14
10949    16
Name: PRODUCT_DEMAND, Length: 10950, dtype: object

#### Обучение и оптимизация модели

Тип данных: табличные данные.

Задача: регрессия.

Опробуем следующие модели: Linear regression, CatBoost, Random forest

In [18]:
# разделение датасета на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [19]:
cat_features = ['PRODUCT_INFORMATIVE_CLASS', 'PRODUCT_INFORMATIVE_CLASS', 'DAY_TYPE', 'MONTH_TYPE']

model_counter = ctb.CatBoostRegressor(cat_features=cat_features, simple_ctr='Counter', n_estimators=100, combinations_ctr='Counter')
model_counter.fit(X_train, y_train)

Learning rate set to 0.3821
0:	learn: 8.2354038	total: 49.4ms	remaining: 4.89s
1:	learn: 6.5077823	total: 52.1ms	remaining: 2.55s
2:	learn: 5.5258051	total: 54.6ms	remaining: 1.76s
3:	learn: 5.1042223	total: 57.2ms	remaining: 1.37s
4:	learn: 4.7573798	total: 59.9ms	remaining: 1.14s
5:	learn: 4.5699730	total: 62ms	remaining: 972ms
6:	learn: 4.2870429	total: 65.2ms	remaining: 866ms
7:	learn: 4.1486754	total: 67.8ms	remaining: 779ms
8:	learn: 3.9750599	total: 70.2ms	remaining: 710ms
9:	learn: 3.9389824	total: 72.4ms	remaining: 652ms
10:	learn: 3.8842514	total: 75ms	remaining: 607ms
11:	learn: 3.8525767	total: 77.6ms	remaining: 569ms
12:	learn: 3.8321131	total: 80.2ms	remaining: 537ms
13:	learn: 3.8107472	total: 82.7ms	remaining: 508ms
14:	learn: 3.7997041	total: 85.3ms	remaining: 484ms
15:	learn: 3.7718366	total: 87.8ms	remaining: 461ms
16:	learn: 3.7598628	total: 90.2ms	remaining: 440ms
17:	learn: 3.7507283	total: 92.7ms	remaining: 422ms
18:	learn: 3.7504890	total: 94.5ms	remaining: 403m

#### Валидация модели и предсказание

In [20]:
preds_counter = model_counter.predict(X_test)



In [21]:
from sklearn.metrics import *

print(mean_squared_error(preds_counter, y_test))


13.999640715168445


#### Формирование заказа с помощью алгоритма